# Sage Combinat Widgets Adapter Tutorial

In [ ]:
# Let's get rid of long error messages for clarity.
from __future__ import print_function
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

Let's say we want to play with a Matrix, ie edit cells, add or drop rows or columns.

In [ ]:
from sage.matrix.constructor import Matrix
m = Matrix(ZZ, 3, 4, range(12))
m

To be able to display this matrix with a grid representation, we need to write an adapter. First, we need to tell the widget which type are our matrix cells, and which would be a default cell content. We will base our adapter on sage_widget_adapters.generic_grid_view_adapter.GridViewAdapter:

In [ ]:
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
class MyAdapter(GridViewAdapter):
    pass

Let's see what happens at this stage:

In [ ]:
from sage_combinat_widgets import GridViewWidget
ma = MyAdapter()
GridViewWidget(m,ma)

We need to tell the widget which cells are to be displayed in the grid view.

In [ ]:
class MyAdapter(GridViewAdapter):
    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

In [ ]:
ma = MyAdapter()
GridViewWidget(m,ma)

We need to tell the widget which type are our matrix cells, and which would be a default cell content (for blank or empty cells).

In [ ]:
class MyAdapter(GridViewAdapter):
    celltype = Integer
    cellzero = Integer(0)

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

Let's try again

In [ ]:
ma = MyAdapter()
GridViewWidget(m,ma)

It works!

## Editing cells

In order to turn our widget into an editable matrix widget, we need accessors for cell content and a method to establish a new matrix value from a new dictionary of cells.

In [ ]:
class MyAdapter(GridViewAdapter):
    celltype = Integer
    cellzero = Integer(0)

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

    @staticmethod
    def get_cell(obj, pos):
        """Return cell content"""
        if pos[0] >= obj.nrows() or pos[1] >= obj.ncols():
            raise ValueError("Entry '%s' does not exist!" % pos)
        return obj[pos[0]][pos[1]]

    @classmethod
    def set_cell(cls, obj, pos, val):
        """Set cell content to `val`"""
        if not val in obj.base_ring():
            raise TypeError("Value '%s' is not compatible!" % val)
        obj.set_block(pos[0], pos[1], matrix(obj.base_ring(), 1, 1, val))
        return obj
    
    @classmethod
    def from_cells(cls, cells={}):
        """Compute a new matrix from a cells dictionary `cells`"""
        nrows = max(pos[0]+1 for pos in cells)
        ncols = max(pos[1]+1 for pos in cells)
        return matrix([[cells[(i,j)] for j in range(ncols)] for i in range(nrows)])


In [ ]:
ma = MyAdapter()
w = GridViewWidget(m,ma)
w

In [ ]:
w.value

In [ ]:
from ipywidgets import HBox, Label
m_input = GridViewWidget(m)
@interact
def f(x = m_input):
    return HBox((GridViewWidget(x),Label('>>>>'), GridViewWidget(x*2)))

## Adding a row